In [1]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import shapiro
from statsmodels.formula.api import ols


In [2]:
CURRENT_DIR = Path.cwd()

In [3]:
filepath = os.path.join(CURRENT_DIR, "S03_residual_stress_merge.xlsx")
dfm = pd.read_excel(filepath).rename(
    columns={"Sigma(x)": "sigma_x_pre", "FWHM": "FWHM_pre"}
)
dfm

,section,sample_no,location,R,W,D,sigma_x_post,FWHM_post,sigma_x_pre,FWHM_pre,diff_sigma_x
0,AA5052,1,1,1400,60,10,13,2.55,-15,2.50,28
1,AA5052,2,1,1400,60,15,16,2.51,2,2.47,14
2,AA5052,3,1,1400,60,20,19,2.47,9,2.48,10
3,AA5052,4,1,1400,70,10,20,2.45,10,2.48,10
4,AA5052,5,1,1400,70,15,6,2.47,0,2.49,6
...,...,...,...,...,...,...,...,...,...,...,...
1130,Center,50,7,1600,70,15,2,2.45,0,0.00,2
1131,Center,51,7,1600,70,20,2,2.45,0,0.00,2
1132,Center,52,7,1600,80,10,5,2.54,0,0.00,5
1133,Center,53,7,1600,80,15,1,2.41,0,0.00,1


In [4]:
# Standardize columnes R, W, D

for col in ["R", "W", "D"]:
    mean = dfm[col].mean()
    std = dfm[col].std()
    dfm[col] = (dfm[col] - mean) / std
dfm

,section,sample_no,location,R,W,D,sigma_x_post,FWHM_post,sigma_x_pre,FWHM_pre,diff_sigma_x
0,AA5052,1,1,-1.225015,-1.222858,-1.222858,13,2.55,-15,2.50,28
1,AA5052,2,1,-1.225015,-1.222858,0.001078,16,2.51,2,2.47,14
2,AA5052,3,1,-1.225015,-1.222858,1.225015,19,2.47,9,2.48,10
3,AA5052,4,1,-1.225015,0.001078,-1.222858,20,2.45,10,2.48,10
4,AA5052,5,1,-1.225015,0.001078,0.001078,6,2.47,0,2.49,6
...,...,...,...,...,...,...,...,...,...,...,...
1130,Center,50,7,1.222858,0.001078,0.001078,2,2.45,0,0.00,2
1131,Center,51,7,1.222858,0.001078,1.225015,2,2.45,0,0.00,2
1132,Center,52,7,1.222858,1.225015,-1.222858,5,2.54,0,0.00,5
1133,Center,53,7,1.222858,1.225015,0.001078,1,2.41,0,0.00,1


## Understanding the ANOVA table columns:

- `df`: Degrees of freedom - the number of independent pieces of information for each source
- `sum_sq`: Sum of squares - the total variation attributed to each factor
- `mean_sq`: Mean square - sum of squares divided by degrees of freedom (variance estimate)
- `F`: F-statistic - the ratio of factor variance to residual variance
- `PR(>F)`: P-value - probability of seeing this F-statistic by chance alone

## What is Type

- typ=1 (Type I): sequential SS, each term tested in the order it appears in the formula.
- typ=2, each term (factor or covariate) is tested after all other main effects but ignoring higher‑order interactions that include it.
  This is usually recommended when the model is reasonably balanced and you want tests that respect the marginality principle (main effects evaluated in the presence of other main effects, but not “penalized” by interactions).
- typ=3 (Type III): each term tested after all other terms including interactions; often used in software like SPSS, especially for unbalanced designs, but interpretation of main effects with strong interactions can be tricky.


In [5]:
# Filter the DataFrame for specific sections
filt = (dfm["section"].isin(["AA5052", "AA6061", "Center"])) 
dfa = dfm[filt]

# Define the formula for the model
formula = "diff_sigma_x ~ R + W + D + section"

# Fit the model
model = ols(formula, data=dfa).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model)

# Print the ANOVA results
anova_results.sort_values(by="PR(>F)")

,df,sum_sq,mean_sq,F,PR(>F)
section,2.0,3.366630e+05,168331.477791,136.497931,8.076070e-54
W,1.0,5.642947e+03,5642.946744,4.575796,3.264068e-02
R,1.0,1.949445e+03,1949.445285,1.580781,2.089082e-01
D,1.0,8.626273e+02,862.627324,0.699494,4.031308e-01
Residual,1129.0,1.392301e+06,1233.216331,NaN,NaN


In [6]:
# Filter the DataFrame for specific sections
filt = dfm["section"].isin(["AA5052"]) 
# filt = dfm["section"].isin(["AA5052"]) & (dfm["location"] == 1)
dfa = dfm[filt]

# Define the formula for the model
base = "diff_sigma_x ~ "
t_main = ["R", "W", "D"]
t_interact_2 = ["R:W", "R:D", "W:D"]
t_interact_3 = ["R:W:D"]
t_single = ["I(R)", "I(W)", "I(D)"]
t_quad = ["I(R**2)", "I(W**2)", "I(D**2)"]
t_quad_pair = ["I(R*W)", "I(R*D)", "I(W*D)"]
t_triple_1 = [
    "I(R**2*W)",
    "I(R**2*D)",
    "I(W**2*R)",
    "I(W**2*D)",
    "I(D**2*R)",
    "I(D**2*W)",
]
t_triple_all = ["I(R*W*D)"]

formula = base + " + ".join(
    t_main
    + t_interact_2
    + t_interact_3
    + t_single
    + t_quad
    + t_quad_pair
    + t_triple_1
    + t_triple_all
)
print(formula)

# Fit the model
model = ols(formula, data=dfa).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model)

# Print the ANOVA results
anova_results.sort_values(by="PR(>F)")
display(anova_results)

coefficients = model.params
print(coefficients)

diff_sigma_x ~ R + W + D + R:W + R:D + W:D + R:W:D + I(R) + I(W) + I(D) + I(R**2) + I(W**2) + I(D**2) + I(R*W) + I(R*D) + I(W*D) + I(R**2*W) + I(R**2*D) + I(W**2*R) + I(W**2*D) + I(D**2*R) + I(D**2*W) + I(R*W*D)


,df,sum_sq,mean_sq,F,PR(>F)
R,1.0,3380.670635,3380.670635,12.720647,0.000410
W,1.0,11.146825,11.146825,0.041943,0.837844
D,1.0,667.063492,667.063492,2.509999,0.114002
R:W,1.0,832.595238,832.595238,3.132855,0.077573
R:D,1.0,224.023810,224.023810,0.842947,0.359168
W:D,1.0,1281.523810,1281.523810,4.822065,0.028733
R:W:D,1.0,6.508929,6.508929,0.024492,0.875728
I(R),1.0,24.460410,24.460410,0.092039,0.761776
I(W),1.0,170.324158,170.324158,0.640889,0.423916
I(D),1.0,31.119314,31.119314,0.117094,0.732407


Intercept        21.676446
R                 1.665003
W                 0.068573
D                 0.740213
R:W              -0.741545
R:D              -0.388565
W:D              -0.921061
R:W:D            -0.065741
I(R)              1.665003
I(W)              0.068573
I(D)              0.740213
I(R ** 2)        -1.007126
I(W ** 2)         1.231252
I(D ** 2)         2.623538
I(R * W)         -0.741545
I(R * D)         -0.388565
I(W * D)         -0.921061
I(R ** 2 * W)     0.279198
I(R ** 2 * D)    -2.266051
I(W ** 2 * R)    -1.045370
I(W ** 2 * D)    -0.551903
I(D ** 2 * R)     0.707735
I(D ** 2 * W)    -0.246733
I(R * W * D)     -0.065741
dtype: float64


In [7]:
# Filter the DataFrame for specific sections
filt = dfm["section"].isin(["AA6061"])
dfa = dfm[filt]

# Define the formula for the model
base = "diff_sigma_x ~ "
t_main = ["R", "W", "D"]
t_interact_2 = ["R:W", "R:D", "W:D"]
t_interact_3 = ["R:W:D"]
t_quad = ["I(R**2)", "I(W**2)", "I(D**2)"]
t_quad_pair = ["I(R*W)", "I(R*D)", "I(W*D)"]
t_triple_1 = [
    "I(R**2*W)",
    "I(R**2*D)",
    "I(W**2*R)",
    "I(W**2*D)",
    "I(D**2*R)",
    "I(D**2*W)",
]
t_triple_all = ["I(R*W*D)"]

formula = base + " + ".join(
    t_main
    + t_interact_2
    + t_interact_3
    + t_quad
    + t_quad_pair
    + t_triple_1
    + t_triple_all
)
print(formula)

# Fit the model
model = ols(formula, data=dfa).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model)

# Print the ANOVA results
anova_results.sort_values(by="PR(>F)")

diff_sigma_x ~ R + W + D + R:W + R:D + W:D + R:W:D + I(R**2) + I(W**2) + I(D**2) + I(R*W) + I(R*D) + I(W*D) + I(R**2*W) + I(R**2*D) + I(W**2*R) + I(W**2*D) + I(D**2*R) + I(D**2*W) + I(R*W*D)


,df,sum_sq,mean_sq,F,PR(>F)
R:W:D,1.0,1.284149e+04,12841.489519,3.855660,0.050343
W,1.0,1.186815e+04,11868.153558,3.563415,0.059866
I(R ** 2 * W),1.0,9.573143e+03,9573.143211,2.874338,0.090862
W:D,1.0,6.569865e+03,6569.865012,1.972603,0.161029
I(D ** 2 * R),1.0,5.528220e+03,5528.220222,1.659849,0.198447
I(D ** 2),1.0,4.681432e+03,4681.432316,1.405601,0.236566
D,1.0,4.466355e+03,4466.355498,1.341024,0.247617
I(W ** 2 * R),1.0,4.019328e+03,4019.328480,1.206804,0.272696
I(W * D),1.0,3.565187e+03,3565.186961,1.070448,0.301535
I(W ** 2),1.0,2.593871e+03,2593.871012,0.778810,0.378091


In [8]:
# Filter the DataFrame for specific sections
filt = dfm["section"].isin(["Center"])
dfa = dfm[filt]

# Define the formula for the model
base = "diff_sigma_x ~ "
t_main = ["R", "W", "D"]
t_interact_2 = ["R:W", "R:D", "W:D"]
t_interact_3 = ["R:W:D"]
t_quad = ["I(R**2)", "I(W**2)", "I(D**2)"]
t_quad_pair = ["I(R*W)", "I(R*D)", "I(W*D)"]
t_triple_1 = [
    "I(R**2*W)",
    "I(R**2*D)",
    "I(W**2*R)",
    "I(W**2*D)",
    "I(D**2*R)",
    "I(D**2*W)",
]
t_triple_all = ["I(R*W*D)"]

formula = base + " + ".join(
    t_main
    + t_interact_2
    + t_interact_3
    + t_quad
    + t_quad_pair
    + t_triple_1
    + t_triple_all
)
print(formula)

# Fit the model
model = ols(formula, data=dfa).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model)

# Print the ANOVA results
anova_results.sort_values(by="PR(>F)")

diff_sigma_x ~ R + W + D + R:W + R:D + W:D + R:W:D + I(R**2) + I(W**2) + I(D**2) + I(R*W) + I(R*D) + I(W*D) + I(R**2*W) + I(R**2*D) + I(W**2*R) + I(W**2*D) + I(D**2*R) + I(D**2*W) + I(R*W*D)


,df,sum_sq,mean_sq,F,PR(>F)
R,1.0,525.777778,525.777778,10.625025,0.001222
W,1.0,315.571429,315.571429,6.377132,0.011987
I(R ** 2 * W),1.0,222.910372,222.910372,4.504618,0.034484
I(R ** 2),1.0,195.047619,195.047619,3.941562,0.047863
R:W:D,1.0,180.035714,180.035714,3.638198,0.057261
D,1.0,91.682540,91.682540,1.852739,0.174314
I(W ** 2 * D),1.0,85.558152,85.558152,1.728977,0.189376
W:D,1.0,81.482143,81.482143,1.646608,0.200244
I(D ** 2),1.0,61.714286,61.714286,1.247135,0.264843
R:W,1.0,52.595238,52.595238,1.062855,0.303255
